# Import Needed Libraries

In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', None)

# Data Cleaning

In [199]:
def property_grouping(p):
    '''
    Given property type, if it is not an apartment, house, condo, townhouse, or loft,
    set as 'Other'
    '''
    properties = ['Apartment', 'House', 'Condomminium', 'Townhouse', 'Loft', 'Other']
    if p not in properties:
        p = 'Other'
    return p

def make_dummies(df):
    '''
    Take column of categories and make dummies in the dataset.
    Drop the original columns
    '''
    columns = list(df.columns)
    for column in columns:
        dummies = pd.get_dummies(df[column])
        df = df.join(dummies)
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('/','_')
    df.columns = df.columns.str.replace('-', '_')
    df.drop(columns=columns,inplace=True)
    return df

In [200]:
#import data as dataframe
df = pd.read_csv('train.csv')

#drop duplicates for Airbnb ID's and drop NA in specific columns
df.drop_duplicates(subset=['id'],inplace=True)
df.dropna(subset=['bathrooms', 'host_has_profile_pic',
                  'host_identity_verified', 'host_since',
                  'bedrooms', 'beds']
                  , inplace=True)

#convert host response rates and review scores to floats
df['host_response_rate'] = df['host_response_rate'].str.strip('%').astype('float')/100
df.review_scores_rating = df.review_scores_rating/100

#fill in empty host response rates and review scores
df['host_response_rate'].fillna(value=0.0, inplace=True)
df['review_scores_rating'].fillna(value=0.0, inplace=True)

#Replace t,f and True, False to 1,0
df.replace({'t':1, 'f': 0}, inplace=True)
df.cleaning_fee = df.cleaning_fee.astype(int)

### Drop unwanted columns

In [201]:
df_cleaned = df.drop(columns = ['id', 'amenities', 'description',
                               'first_review', 'host_since', 'last_review',
                               'name', 'neighbourhood', 'thumbnail_url',
                                'zipcode', 'city'])

In [202]:
df_cleaned['property_type'] = df_cleaned['property_type'].apply(property_grouping)

In [203]:
df_cleaned = df_cleaned[df_cleaned['cancellation_policy'].str.contains('super')==False]

In [204]:
df_cleaned.head()

,log_price,property_type,room_type,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,host_has_profile_pic,host_identity_verified,host_response_rate,instant_bookable,latitude,longitude,number_of_reviews,review_scores_rating,bedrooms,beds
0,5.010635,Apartment,Entire home/apt,3,1.0,Real Bed,strict,1,1,1,0.0,0,40.696524,-73.991617,2,1.00,1.0,1.0
1,5.129899,Apartment,Entire home/apt,7,1.0,Real Bed,strict,1,1,0,1.0,1,40.766115,-73.989040,6,0.93,3.0,3.0
2,4.976734,Apartment,Entire home/apt,5,1.0,Real Bed,moderate,1,1,1,1.0,1,40.808110,-73.943756,10,0.92,1.0,3.0
3,6.620073,House,Entire home/apt,4,1.0,Real Bed,flexible,1,1,1,0.0,0,37.772004,-122.431619,0,0.00,2.0,2.0
4,4.744932,Apartment,Entire home/apt,2,1.0,Real Bed,moderate,1,1,1,1.0,1,38.925627,-77.034596,4,0.40,0.0,1.0


# Analysis

In [205]:
X_numeric = df_cleaned.select_dtypes(exclude='object')
X_cat = df_cleaned.select_dtypes(include='object')

In [206]:
X_cat = make_dummies(X_cat)

In [207]:
#X_cat.drop(columns=['Apartment', 'Private_room', 'Real_Bed', 'moderate'], inplace=True)

In [208]:
print(X_numeric.shape, X_cat.shape)

(73455, 14) (73455, 16)


In [209]:
X = X_numeric.join(X_cat)

In [210]:
y = X.log_price
X.drop(columns='log_price',inplace=True)

In [211]:
X.head()

,accommodates,bathrooms,cleaning_fee,host_has_profile_pic,host_identity_verified,host_response_rate,instant_bookable,latitude,longitude,number_of_reviews,review_scores_rating,bedrooms,beds,Apartment,House,Loft,Other,Townhouse,Entire_home_apt,Private_room,Shared_room,Airbed,Couch,Futon,Pull_out_Sofa,Real_Bed,flexible,moderate,strict
0,3,1.0,1,1,1,0.0,0,40.696524,-73.991617,2,1.00,1.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
1,7,1.0,1,1,0,1.0,1,40.766115,-73.989040,6,0.93,3.0,3.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
2,5,1.0,1,1,1,1.0,1,40.808110,-73.943756,10,0.92,1.0,3.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0
3,4,1.0,1,1,1,0.0,0,37.772004,-122.431619,0,0.00,2.0,2.0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4,2,1.0,1,1,1,1.0,1,38.925627,-77.034596,4,0.40,0.0,1.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0


In [212]:
lr = LinearRegression()
lr.fit(X,y)
lr.score(X,y)

0.5376585119551911

In [213]:
df.log_price.describe()

count    73579.000000
mean         4.782890
std          0.717937
min          0.000000
25%          4.317488
50%          4.718499
75%          5.220356
max          7.600402
Name: log_price, dtype: float64

In [214]:
formula = 'log_price ~ ' + '+'.join(X.columns)

In [215]:
formula

'log_price ~ accommodates+bathrooms+cleaning_fee+host_has_profile_pic+host_identity_verified+host_response_rate+instant_bookable+latitude+longitude+number_of_reviews+review_scores_rating+bedrooms+beds+Apartment+House+Loft+Other+Townhouse+Entire_home_apt+Private_room+Shared_room+Airbed+Couch+Futon+Pull_out_Sofa+Real_Bed+flexible+moderate+strict'

In [216]:
import statsmodels.formula.api as smf

mod = smf.ols(formula=formula, data = X.join(y))
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.538
Model:                            OLS   Adj. R-squared:                  0.538
Method:                 Least Squares   F-statistic:                     3416.
Date:                Mon, 02 Mar 2020   Prob (F-statistic):               0.00
Time:                        18:23:26   Log-Likelihood:                -51482.
No. Observations:               73455   AIC:                         1.030e+05
Df Residuals:                   73429   BIC:                         1.033e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.9204      0.036     25.266      0.000       0.849       0.992
accommodates               0.0820      0.002     49.593      0.000       0.079       0.085
bathrooms                  0.1461      0.004     36.620      0.000       0.138       0.154
cleaning_fee              -0.0223      0.005     -4.897      0.000      -0.031      -0.013
host_has_profile_pic      -0.0777      0.033     -2.371      0.018      -0.142      -0.013
host_identity_verified    -0.0040      0.004     -0.981      0.327      -0.012       0.004
host_response_rate        -0.0998      0.005    -20.435      0.000      -0.109      -0.090
instant_bookable          -0.0486      0.004    -11.612      0.000      -0.057      -0.040
latitude                   0.0481      0.001     35.998      0.000       0.046       0.051
longitude                 -0.0059      0.000    -31.239      0.000      -0.006      -0.006
number_of_reviews      -3.607e-05   5.17e-05     -0.698      0.485      -0.000    6.53e-05
review_scores_rating      -0.1653      0.005    -31.720      0.000      -0.175      -0.155
bedrooms                   0.1479      0.003     42.950      0.000       0.141       0.155
beds                      -0.0505      0.003    -19.134      0.000      -0.056      -0.045
Apartment                  0.1311      0.008     15.439      0.000       0.114       0.148
House                      0.1214      0.009     14.028      0.000       0.104       0.138
Loft                       0.3079      0.014     22.355      0.000       0.281       0.335
Other                      0.2122      0.010     21.147      0.000       0.192       0.232
Townhouse                  0.1478      0.012     12.019      0.000       0.124       0.172
Entire_home_apt            0.8878      0.013     68.481      0.000       0.862       0.913
Private_room               0.2564      0.013     19.857      0.000       0.231       0.282
Shared_room               -0.2238      0.014    -15.748      0.000      -0.252      -0.196
Airbed                     0.1175      0.021      5.569      0.000       0.076       0.159
Couch                      0.2797      0.027     10.519      0.000       0.228       0.332
Futon                      0.1204      0.018      6.654      0.000       0.085       0.156
Pull_out_Sofa              0.2202      0.020     11.236      0.000       0.182       0.259
Real_Bed                   0.1825      0.011     15.996      0.000       0.160       0.205
flexible                   0.2947      0.012     23.833      0.000       0.270       0.319
moderate                   0.2934      0.013     23.357      0.000       0.269       0.318
strict                     0.3324      0.012     26.671      0.000       0.308       0.357
==============================================================================
O